In [49]:
import requests
import pandas as pd
from io import BytesIO

# E_抓取檔案
## requests.get下載檔案
url = "https://www.ris.gov.tw/info-popudata/app/awFastDownload/file/y0s6-00000.xls/y0s6/00000/"
res = requests.get(url=url)

## 連結本身就會下載excel檔案，下載完後將其讀入
excel_data = BytesIO(res.content)
df = pd.read_excel(excel_data)

In [50]:
import pandas as pd

# T_修正欄位名稱
columns = ["location", "population", "area(km^2)", "density"]
df.columns = columns

In [51]:
import pandas as pd

# T_製作副本，並將「縣市」列表取出做成list
df_city_list = df.copy()
df_city_list = df_city_list.iloc[3:29]

df_city_list["location"] = df_city_list["location"].str.replace(" ", "").str.replace("　", "").str.replace("※", "")

city_list = list(df_city_list["location"])

In [52]:
import pandas as pd

# T_對總表修改欄位，去除掉多餘欄位（僅保留縣市及鄉鎮市區資料）
df.drop(index=range(0,42), axis=0, inplace=True)
df["location"] = df["location"].str.replace(" ", "").str.replace("　", "").str.replace("※", "")
df = df.reset_index(drop=True)

In [53]:
import pandas as pd

# T_根據前面取得的縣市列表，取得「縣市」整區資料的位置並做成list
city_index = []
for city in city_list:
    idx = df.index[df["location"] == city].tolist()
    city_index.append(int(idx[0]))

In [54]:
import pandas as pd

# T_新增一欄"city"並根據剛剛取得的縣市位置，將縣市填入，取得縣市+區兩個欄位
df["city"] = None

for n in range(0, len(city_index)):
    if n < len(city_index) - 1:
        df.loc[city_index[n]:city_index[n+1]-1, "city"] = city_list[n]
    else:
        df.loc[city_index[n]:, "city"] = city_list[n]

In [55]:
import pandas as pd

# T_重新調整欄位名並重置index確保其索引正確
new_columns = ["district", "population", "area(km^2)", "density", "city"]
df.columns = new_columns
new_order = ["city", "district", "population", "area(km^2)", "density"]
df = df[new_order]

In [56]:
import pandas as pd

# T_去除人口及人口密度（資料為去年底的過時資料）
df.drop(columns=["population", "density"], axis=1, inplace=True)

In [57]:
import pandas as pd

# T_將面積四捨五入至小數點後第二位
df["area(km^2)"] = df["area(km^2)"].apply(float)
df["area(km^2)"] = df["area(km^2)"].round(2)

In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df.to_sql(name="raw_location", con=engine, if_exists="replace", index=True)

print("資料已輸入資料庫！")

394